# Tugas Klasifikasi dan Prediksi
**Data Mining and Data Warehouse**

### Kelompok 3
- Ayesha Ali Firdaus (2101990)
- Ibnu Adeng Kurnia (2101769)
- Ihsan Ghozi Zulfikar (2103303)
- M. Fadhillah Nursyawal (2107135)
- M. Labib Husain (2101989)

In [123]:
# Impotr Library
%matplotlib inline
import numpy as np
import joblib
import pandas as pd
import datetime
from datetime import date
import pickle
from pandas.api.types import CategoricalDtype

# Import Data Set
df = pd.read_csv("C:\\Users\\AYESHA ALI FIRDAUS\\DatMin\\orders_class.csv")

## ==========Praproses===========

In [124]:
# Mengambil struktur data train dan konversi tipe atribut nominal menjadi kategori
df_train = pd.read_csv("C:\\Users\\AYESHA ALI FIRDAUS\\DatMin\\orders_train.csv")
cat_col = ["salutation", 'state', 'color', 'size']
for col in cat_col: 
    df_train[col] = df_train[col].astype("category") 


In [125]:
# ================= DF (order_train) ================================
# Dibuat null karena tanggal lahir tidak bisa di tebak
df.loc[df.dateOfBirth == "?", "dateOfBirth"] = np.nan
df.loc[(df.dateOfBirth == "1655-04-19", "dateOfBirth")] = np.nan

# Ingin mengubah deiveryDate yang anomali menjadi median dari waktu antar 
df.loc[df.deliveryDate == "?", "deliveryDate"] = np.nan #-------------- Menjadikan NULL terlebih dahulu data yang "?"
df.loc[df.deliveryDate < "2012", "deliveryDate"] = np.nan #------------ Mengubah menjadi NULL data yang kurang dari 2012
df['orderDate'] = pd.to_datetime(df['orderDate']) #-------------------- Ubah tipe data menjadi date agar bisa menghitung hari
df['deliveryDate'] = pd.to_datetime(df['deliveryDate']) #-------------- Ubah tipe data menjadi date agar bisa menghitung hari
df['handling_time'] = df['deliveryDate'] - df['orderDate'] #----------- Menambahkan atribut total hari pengantaran dari data yang tidak anomali {di dapat dari mengurangi tanggal pengiriman dengan tanggal order}
df.loc[df.handling_time < "0 days", "handling_time"] = np.nan #-------- Menjadikan NULL data yang anomali akibat deliveryDate yang anomali {contoh: 1990}
df['handling_time'].fillna(df['handling_time'].median(),inplace=True) # Mengisikan handling time yang NULL dengan median
df['deliveryDate'] = df['orderDate'] + df['handling_time'] #----------- Semua delivery date yang "?" dan kurang dari 2012 ditambahin median dari handling_time

# Menambahkan atribut age ke dalam dataset
df['dateOfBirth'] = pd.to_datetime(df['dateOfBirth']) #------------------- Mengubah tipe data dateOfBirth menjadi date
df['age'] = datetime.datetime(2013, 12, 31, 0, 0, 0) - df['dateOfBirth'] # Menghitung umur dari tahun 2013 akhir dikurangi dengan tanggal lahir
df['age'] = df['age'].dt.days #------------------------------------------- Mengubah tipe data date menjadi float64
df["age"] = df["age"].astype("Int64") #----------------------------------- Mengubah tipe data float64 menjadi int64
df["age"] = df["age"]  / 365 #-------------------------------------------- Mengubah hari menjadi tahun dengan dibagi oleh 365 hari
df["age"] = df["age"].astype("Int64") #----------------------------------- Mengubah tipe data float64 menjadi int64 {akibat dari hasil pembagian sebelumnya}
df['age'].fillna(df['age'].median(),inplace=True) #----------------------- Mengisikan age yang NULL dengan median dari age

# Mengubah tipe data salutation menjadi category
df["salutation"] = df["salutation"].astype("category")

# Mengubah handling time menjadi int
df['handling_time'] = df['handling_time'].dt.days #------------------------------------------- Mengubah tipe data date menjadi float64
df["handling_time"] = df["handling_time"].astype("Int64")

In [126]:
# Mengkonversi data yang akan dites agar sama dengan data train
cat_col = ["salutation", 'state', 'color', 'size']
for col in cat_col: 
    cat_type = CategoricalDtype(categories=df_train[col].cat.categories)
    df[col] = df[col].astype(cat_type)

In [127]:
# Melakukan drop atribut yang tidak perlu, agar sama dengan data train
df_temp = df.drop(['orderItemID', 'dateOfBirth','creationDate', 'orderDate', 'deliveryDate'],axis=1)

# Melakukan One-Hot Encoding
df_temp = pd.get_dummies(data=df_temp, columns = ['state', 'color', 'size', 'salutation'])

In [128]:
# Membaca file pickle yang sudah dibuat
with open ("C:\\Users\\AYESHA ALI FIRDAUS\\DatMin\\orders_class.pickle", 'rb') as fp:
    X_train_column = list(pickle.load(fp))
    
df_temp = df_temp[X_train_column]

In [130]:
# Mengambil hasil model dari XGBoost sebelumnya
clf = joblib.load("C:\\Users\\AYESHA ALI FIRDAUS\\DatMin\\xgboost_order_class.joblib")

# Melakukan prediksi pada data test
df_hasil = pd.DataFrame()
df_hasil['orderItemID'] = df['orderItemID']
df_hasil['prediction'] = clf.predict(df_temp)

# Mengekspor data test
df_hasil.to_csv('C:\\Users\\AYESHA ALI FIRDAUS\\DatMin\\order_class.txt', sep=';', index=False)